In [102]:
import yaml
import json
import os
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output


In [103]:
!rm scratch.jsonl

In [104]:
!cat scratch.jsonl

cat: scratch.jsonl: No such file or directory


In [124]:

styles = {
    'Accepted': 'background-color: rgba(0, 255, 0, 0.3);',  # Translucent green
    'Rejected': 'background-color: rgba(255, 0, 0, 0.3);',  # Translucent red
    'No judgement': 'background-color: rgba(0, 0, 0, 0.3);'
}

def display_item(data, judgement_file, index=0):
    if os.path.isfile(judgement_file):
        with open(judgement_file) as fle:
            judgement_map = yaml.safe_load(fle)
    else:
        judgement_map = dict()

    clear_output(wait=True)
    item = data[index]
    text_display = Markdown(item['text'])

    accept_button = widgets.Button(description="Accept")
    reject_button = widgets.Button(description="Reject")
    next_button = widgets.Button(description="Next")
    prev_button = widgets.Button(description="Previous")

    def display_verdict(verdict):
        verdict_display = HTML(f"<div style='{styles[verdict]}'>{verdict}</div>")
        display(verdict_display)

    def on_accept(b):
        judgement_map[item["id"]] = True
        with open(judgement_file, 'w') as fle:
            yaml.dump(judgement_map, fle)

        navigate(1)

    def on_reject(b):
        judgement_map[item["id"]] = False
        with open(judgement_file, 'w') as fle:
            yaml.dump(judgement_map, fle)
        navigate(1)

    def navigate(step):
        nonlocal index
        index = min(max(0, index + step), len(data) - 1)
        display_item(data, judgement_file, index)

    def display_buttons():
        button_box = widgets.HBox([accept_button, reject_button, prev_button, next_button])
        display(button_box)
    
    def display_location(index):
        display(Markdown(f"Index: {index}/{len(data)}"))

        display(Markdown(f"Post ID: {item['id']}"))
    
        num_judged = len(judgement_map.keys())
        num_accepted = sum(judgement_map.values())
        display(Markdown(f"Accepted: {num_accepted}, Judged: {num_judged}"))

    accept_button.on_click(on_accept)
    reject_button.on_click(on_reject)
    next_button.on_click(lambda b: navigate(1))
    prev_button.on_click(lambda b: navigate(-1))

    display_buttons()

    if item["id"] in judgement_map:
        verdict = 'Accepted' if judgement_map[item["id"]] else 'Rejected'
    else:
        verdict = 'No judgement'

    display(text_display)
    display_verdict(verdict)
    display_location(index)

# Example data and display call
data = [{'id': 47, 'text': 'Some MathJax content $E=mc^2$'}, 
        {'id': 132, 'text': 'More content $\int_0^\infty e^{-x^2} dx$'},
        {'id': 12, 'text': 'More filler content'}]
display_item(data, "scratch.jsonl")


Some MathJax content $E=mc^2$

Index: 0/3

Post ID: 47

Accepted: 0, Judged: 0

In [125]:
def load_stackexchange(filepath):
    data = []
    with open(filepath) as fle:
        for line in fle:
            row = json.loads(line)
            row["text"] = "---INPUT:\n\n" + row["input"] + "\n\n---OUTPUT:\n" + row["output"]
            row["id"] = row["meta"]["post_id"]
            data.append(row)
    return data

In [129]:
!cat human-judgements/cstheory_judgements.yaml
!wc -l human-judgements/cstheory_judgements.yaml

41: false
159: false
167: false
175: true
184: false
284: true
376: false
524: true
529: true
562: true
682: false
710: false
716: false
761: false
799: true
914: true
1026: false
1046: false
1063: false
1233: false
1263: false
1298: true
1313: true
1348: false
1521: false
1824: false
1893: false
1948: false
2021: false
2045: true
2149: false
2229: true
2252: false
2277: false
2315: true
2328: false
2349: true
2353: true
2426: true
2428: true
2434: true
2461: false
2505: true
2585: false
2703: false
2800: false
2892: false
3074: true
3287: false
3304: true
3332: false
3360: false
3496: false
3786: false
3863: false
3888: false
3987: true
4027: false
4096: true
4489: false
4683: false
4697: false
4746: true
4816: true
4882: false
5003: false
5018: false
5110: true
5157: true
5158: true
5245: true
5395: true
5427: false
5431: false
5922: true
5994: false
6085: true
6519: false
6713: true
6735: false
6748: true
6753: false
6952: false
6959: false
7029: true
7129: true
7388: false
7469: fa

In [ ]:
data_file = "filtered-stack-exchange/cstheory_filtered.jsonl"
judgement_file = "human-judgements/cstheory_judgements.yaml"
se_data = load_stackexchange(data_file)
print(data)

display_item(se_data, judgement_file=judgement_file, index=210)

---INPUT:

Scott Aaronson proposed an interesting challange: can we use supercomputers today to help solve CS problems in the same way that physicists use large particle colliders?

More concretely, my proposal is to
  devote some of the world’s computing
  power to an all-out attempt to answer
  questions like the following: does
  computing the permanent of a 4-by-4
  matrix require more arithmetic
  operations than computing its
  determinant?

He concludes that this would require ~$10^{123}$ floating point operations, which is beyond our current means.  The slides are available and are also worth reading.  
Is there any precedence for solving open TCS problems through brute force experimentation?

---OUTPUT:
In "Finding Efficient Circuits Using SAT-solvers", Kojevnikov, Kulikov, and Yaroslavtsev have used SAT solvers to find better circuits for computing $MOD_k$ function. 
I have used computers to find proofs of time-space lower bounds, as described here. But that was only feasible because I was working with an extremely restrictive proof system.
Maverick Woo and I have been working for some time to find the "right" domain for proving circuit upper/lower bounds using computers. We had hoped that we may resolve $CC^0$ vs $ACC^0$ (or a very weak version of it) using SAT solvers, but this is looking more and more unlikely. (I hope Maverick doesn't mind me saying this...)
The first generic problem with using brute-force search to prove nontrivial lower bounds is that it just takes too damn long, even on a very fast computer. The alternative is to try to use SAT solvers, QBF solvers, or other sophisticated optimization tools, but they do not seem to be enough to offset the enormity of the search space. Circuit synthesis problems are among the hardest practical instances one can come by.
The second generic problem is that the "proof" of the resulting lower bound (obtained by running brute-force search and finding nothing) would be insanely long and apparently yield no insight (other than the fact that the lower bound holds). So a big challenge to "experimental complexity theory" is to find interesting lower bound questions for which the eventual "proof" of the lower bound is short enough to be verifiable, and interesting enough to lead to further insights.

Index: 7/213

Post ID: 524

Accepted: 96, Judged: 213